In [1]:
%gui qt
%load_ext autoreload
%autoreload 2
    
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.Bnd import Bnd_OBB, Bnd_Box
from OCC.Core.BRepBuilderAPI import (
    BRepBuilderAPI_Transform,
    BRepBuilderAPI_MakeEdge,
    BRepBuilderAPI_MakeWire,
    BRepBuilderAPI_MakeFace,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopoDS import TopoDS_Compound, TopoDS_Wire, TopoDS_Vertex, TopoDS_Shape, TopoDS_Face
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.gp import gp_Trsf, gp_Pnt, gp_Ax3, gp_Ax1, gp_Dir, gp_Pln, gp_Vec, gp_Mat, gp_Quaternion
from OCC.Display.SimpleGui import init_display
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from IPython.display import display
from IPython.display import display, HTML
from math import acos, degrees
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.Geom import Geom_CylindricalSurface


import pandas as pd
import numpy as np
import json
import math

# Start display without blocking
viewer, start_display, add_menu, add_function_to_menu = init_display()

# JSON Library path
json_path = "../data/Shape_classifier_info.json"

viewer.EraseAll()
viewer.View_Iso()
viewer.FitAll()

pyside6 backend - Qt version 6.8.3


In [2]:
# Load the STEP and extract 'solid'
def load_step(file_path: str):
    reader = STEPControl_Reader()
    status = reader.ReadFile(file_path)
    if status != 1:
        raise RuntimeError(f"Failed to read STEP file: {file_path}")
    reader.TransferRoots()
    return reader.Shape()

In [3]:
def visualize(shape, display, color="CYAN", clear=False):
    """
    Displays a shape with global XYZ axes and OBB extents.
    
    Args:
        shape: The TopoDS_Shape to display.
        display: OCC display handle (e.g., from init_display()).
        color: Optional color for the shape.
        clear: If True, erase all previous shapes before displaying.
    """

    def get_occ_color(color_name):
        colors = {
            "RED": (1.0, 0.0, 0.0),
            "GREEN": (0.0, 1.0, 0.0),
            "BLUE": (0.0, 0.0, 1.0),
            "PSS_DARK_BLUE": (0.023, 0.105, 0.215),
            "PSS_LIGHT_BLUE": (0.592, 0.792, 0.921),
            "YELLOW": (1.0, 1.0, 0.0),
            "ORANGE": (1.0, 0.5, 0.0),
            "CYAN": (0.0, 1.0, 1.0),
            "MAGENTA": (1.0, 0.0, 1.0),
            "WHITE": (1.0, 1.0, 1.0),
            "GRAY": (0.5, 0.5, 0.5),
            "BLACK": (0.0, 0.0, 0.0),
        }
        r, g, b = colors.get(color_name.upper(), (1.0, 1.0, 1.0))
        return Quantity_Color(r, g, b, Quantity_TOC_RGB)
    
    # if clear:
        viewer.EraseAll()

    # Convert color name to Quantity_Color
    qcolor = get_occ_color(color)

    # Display the shape
    viewer.DisplayShape(shape, color=qcolor, update=False)

    viewer.FitAll()



In [4]:
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def draw_obb_box(obb, viewer, color="ORANGE", update=True):
    """
    Draws the 12 edges of the OBB as lines in the viewer.
    """
    c = obb['center']
    cx, cy, cz = c.X(), c.Y(), c.Z()
    hx, hy, hz = obb['half_lengths'][0], obb['half_lengths'][1], obb['half_lengths'][2]
    dx, dy, dz = obb['dir_x'], obb['dir_y'], obb['dir_z']

    # Precompute corner offsets
    offsets = [
        (+hx, +hy, +hz), (-hx, +hy, +hz), (-hx, -hy, +hz), (+hx, -hy, +hz),
        (+hx, +hy, -hz), (-hx, +hy, -hz), (-hx, -hy, -hz), (+hx, -hy, -hz)
    ]

    # Compute corners
    corners = []
    for ox, oy, oz in offsets:
        x = cx + ox * dx.X() + oy * dy.X() + oz * dz.X()
        y = cy + ox * dx.Y() + oy * dy.Y() + oz * dz.Y()
        z = cz + ox * dx.Z() + oy * dy.Z() + oz * dz.Z()
        corners.append(gp_Pnt(x, y, z))

    # Define edges by corner indices
    edges_idx = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # top face
        (4, 5), (5, 6), (6, 7), (7, 4),  # bottom face
        (0, 4), (1, 5), (2, 6), (3, 7),  # vertical edges
    ]

    for i1, i2 in edges_idx:
        edge = BRepBuilderAPI_MakeEdge(corners[i1], corners[i2]).Edge()
        viewer.DisplayShape(edge, color=color, update=False)

    if update:
        viewer.FitAll()

In [5]:
# # face alignemnt worst case fix, no z/x/y alignment.
# # Find the largest face by area → assume it defines primary orientation.
# # Align its longest edge to the Z axis.
# # Find a second large face roughly orthogonal to that → use to resolve X or Y.

# from OCC.Core.TopExp import TopExp_Explorer
# from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
# from OCC.Core.BRepGProp import brepgprop
# from OCC.Core.GProp import GProp_GProps
# from OCC.Core.BRep import BRep_Tool
# from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
# from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform


# def robust_align_solid_from_geometry(solid, tol=1e-3):
#     """
#     Align a solid using its geometry:
#     - Z axis is taken from the longest edge of the largest planar face
#     - X/Y axes are derived from a second large face roughly perpendicular to Z
#     Returns: aligned solid, transformation, and reference Ax3
#     """
#     # Step 1: Find the largest planar face
#     explorer = TopExp_Explorer(solid, TopAbs_FACE)
#     largest_face = None
#     max_area = 0.0
#     face_areas = []

#     while explorer.More():
#         face = explorer.Current()
#         props = GProp_GProps()
#         brepgprop.SurfaceProperties(face, props)
#         area = props.Mass()
#         face_areas.append((face, area))
#         if area > max_area:
#             max_area = area
#             largest_face = face
#         explorer.Next()

#     if not largest_face:
#         raise RuntimeError("No faces found in solid.")

#     # Step 2: Get longest edge of that face
#     edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
#     longest_edge = None
#     max_length = 0.0
#     longest_vec = None

#     while edge_exp.More():
#         edge = edge_exp.Current()
#         curve_handle, u1, u2 = BRep_Tool.Curve(edge)
#         curve = BRepAdaptor_Curve(edge)
#         p1 = curve.Value(curve.FirstParameter())
#         p2 = curve.Value(curve.LastParameter())
#         vec = gp_Vec(p1, p2)
#         length = vec.Magnitude()
#         if length > max_length:
#             max_length = length
#             longest_edge = edge
#             longest_vec = vec
#         edge_exp.Next()

#     if not longest_vec:
#         raise RuntimeError("No edge found on largest face.")

#     z_dir = gp_Dir(longest_vec)

#     # Step 3: Find a second large face with normal ≠ Z to define X
#     second_face = None
#     second_normal = None
#     for face, area in sorted(face_areas, key=lambda x: -x[1]):
#         if face.IsSame(largest_face):
#             continue
#         u, v = 0.5, 0.5
#         surf_adapt = BRepAdaptor_Surface(face)
#         normal = surf_adapt.Plane().Axis().Direction() if surf_adapt.GetType() == 0 else None
#         if normal and abs(normal.Dot(z_dir)) < 0.9:  # roughly perpendicular
#             second_face = face
#             second_normal = normal
#             break

#     if not second_normal:
#         raise RuntimeError("No suitable secondary face found for X/Y orientation.")

#     x_dir = second_normal
#     y_dir = z_dir.Crossed(x_dir)

#     # Step 4: Create transformation from current CS to global
#     origin = gp_Pnt(0, 0, 0)
#     from_cs = gp_Ax3(origin, z_dir, x_dir)
#     to_cs = gp_Ax3(origin, gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))

#     trsf = gp_Trsf()
#     trsf.SetDisplacement(from_cs, to_cs)

#     aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()
#     return aligned, trsf, to_cs, second_face

In [6]:
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_EDGE
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_Surface
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Vec, gp_Ax3, gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def robust_align_solid_from_geometry(solid, tol=1e-3):
    """
    Aligns a solid such that:
    - X axis is the longest edge on the largest planar face
    - Y axis is the normal of that face
    - Z axis is X × Y (right-handed)
    Returns:
    - aligned solid
    - transformation used
    - local coordinate system (gp_Ax3)
    - largest face used
    - dir_x, dir_y, dir_z
    """
    # Step 1: Find the largest planar face
    explorer = TopExp_Explorer(solid, TopAbs_FACE)
    largest_face = None
    max_area = 0.0

    while explorer.More():
        face = explorer.Current()
        props = GProp_GProps()
        brepgprop.SurfaceProperties(face, props)
        area = props.Mass()
        if area > max_area:
            max_area = area
            largest_face = face
        explorer.Next()

    if not largest_face:
        raise RuntimeError("No planar face found in solid.")

    # Step 2: Get longest edge on that face → X axis
    edge_exp = TopExp_Explorer(largest_face, TopAbs_EDGE)
    longest_vec = None
    max_length = 0.0

    while edge_exp.More():
        edge = edge_exp.Current()
        curve = BRepAdaptor_Curve(edge)
        p1 = curve.Value(curve.FirstParameter())
        p2 = curve.Value(curve.LastParameter())
        vec = gp_Vec(p1, p2)
        length = vec.Magnitude()
        if length > max_length:
            max_length = length
            longest_vec = vec
        edge_exp.Next()

    if not longest_vec or longest_vec.Magnitude() < tol:
        raise RuntimeError("Failed to find valid longest edge.")

    dir_x = gp_Dir(longest_vec)

    # Step 3: Get face normal → Y axis
    surf_adapt = BRepAdaptor_Surface(largest_face)
    if surf_adapt.GetType() != 0:  # not a plane
        raise RuntimeError("Largest face is not planar.")
    dir_y = surf_adapt.Plane().Axis().Direction()

    # Step 4: Compute Z = X × Y, and re-orthogonalize Y to X and Z
    dir_z = dir_x.Crossed(dir_y)
    dir_y = dir_z.Crossed(dir_x)  # ensure orthogonality

    # Step 5: Create transformation from this local frame to global (X,Y,Z)
    origin = gp_Pnt(0, 0, 0)
    from_cs = gp_Ax3(origin, dir_x, dir_y)
    to_cs = gp_Ax3(origin, gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))  # global X/Y/Z

    trsf = gp_Trsf()
    trsf.SetDisplacement(from_cs, to_cs)

    aligned = BRepBuilderAPI_Transform(solid, trsf, True).Shape()

    return aligned, trsf, to_cs, largest_face, dir_x, dir_y, dir_z


In [7]:
# from OCC.Core.gp import gp_Ax3, gp_Trsf, gp_Pnt, gp_Dir
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

# def align_to_dstv_frame(solid, origin_local, dir_x, dir_y, dir_z):
#     """
#     Transforms a solid so that:
#     - dir_x aligns to global X (beam length)
#     - dir_y aligns to global Y (beam height)
#     - dir_z aligns to global Z (beam width)
#     - origin_local maps to global (0, 0, 0)
#     """
#     # Create local coordinate system from solid geometry
#     local_ax3 = gp_Ax3(origin_local, dir_z, dir_x)  # Z = normal, X = length

#     # Create global DSTV coordinate system: Z = up, X = length, Y = height
#     global_ax3 = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))

#     # Create transformation
#     trsf = gp_Trsf()
#     trsf.SetDisplacement(local_ax3, global_ax3)

#     # Apply to shape
#     transformer = BRepBuilderAPI_Transform(solid, trsf, True)
#     return transformer.Shape(), trsf


In [8]:
from OCC.Core.gp import gp_Ax3, gp_Trsf, gp_Pnt, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_to_dstv_frame(solid, origin_local, dir_x, dir_y, dir_z):
    """
    Transforms a solid so that:
    - dir_x aligns to global X (beam length)
    - dir_y aligns to global Y (beam height)
    - dir_z aligns to global Z (beam width)
    - origin_local maps to global (0, 0, 0)
    """
    # Create local coordinate system from solid geometry
    local_ax3 = gp_Ax3(origin_local, gp_Dir(*dir_x), gp_Dir(*dir_y))

    # Create global DSTV coordinate system
    global_ax3 = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(1, 0, 0), gp_Dir(0, 1, 0))

    # Create transformation
    trsf = gp_Trsf()
    trsf.SetDisplacement(local_ax3, global_ax3)

    # Apply to shape
    transformer = BRepBuilderAPI_Transform(solid, trsf, True)
    return transformer.Shape(), trsf


In [9]:
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir

def compute_dstv_origin(center: gp_Pnt, extents: list, dir_x: gp_Dir, dir_y: gp_Dir, dir_z: gp_Dir):
    """
    Computes the DSTV origin (rear-bottom-left) for a shape aligned via OBB.

    Parameters:
    - center: gp_Pnt — the center of the OBB (post-alignment)
    - extents: [length, height, width] — full extents along dir_x, dir_y, dir_z
    - dir_x: gp_Dir — length/feed direction
    - dir_y: gp_Dir — height/web direction
    - dir_z: gp_Dir — width/flange direction

    Returns:
    - gp_Pnt — rear-bottom-left corner in the aligned local frame
    """

    # Compute offset from center to rear-bottom-left corner
    dx = -extents[0] / 2  # back along length
    dy = -extents[1] / 2  # down along height
    dz = -extents[2] / 2  # left along width

    # Offset vector from center to origin
    offset_vec = gp_Vec(dir_x).Scaled(dx) + gp_Vec(dir_y).Scaled(dy) + gp_Vec(dir_z).Scaled(dz)

    # Translate center to origin
    origin_local = center.Translated(offset_vec)
    return origin_local


In [10]:
# # classify shape to JSON profile list
# def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
#     """
#     Attempts to classify a structural profile by matching its dimensions and area.
#     Tries both the original and swapped width/height to handle OBB axis flips.
#     """
#     with open(json_path) as f:
#         lib = json.load(f)
#     def try_match(height, width):
#         best, best_score = None, float('inf')
#         for cat, ents in lib.items():
#             for name, info in ents.items():
#                 dh = abs(height - info["height"])
#                 dw = abs(width  - info["width"])
#                 if dh > tol_dim or dw > tol_dim:
#                     continue
#                 ea = abs(cs["area"] - info["csa"]) / info["csa"]
#                 if ea > tol_area:
#                     continue
#                 el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
#                 score = dh + dw + ea * 100 + el * 100
#                 if score < best_score:
#                     best_score = score
#                     best = {
#                         **info,
#                         "Designation": name,
#                         "Category": cat,
#                         "Measured_height": height,
#                         "Measured_width": width,
#                         "Measured_area": cs["area"],
#                         "Measured_length": cs["length"],
#                         "Match_score": score,
#                         "Profile_type": info["code_profile"],
#                         "json_height": info["height"],
#                         "json_width": info["width"],
#                         "json_csa": info["csa"],
#                     }
#         return best, best_score

#         print(f"Best: {best}")
#         print(f"Best_score: {best_score}")
    
#     # Try both normal and swapped width/height
#     original, score1 = try_match(cs["span_web"], cs["span_flange"])
#     swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

#     if original and (not swapped or score1 <= score2):
#         original["Swapped_dimensions"] = False
#         return original
#     elif swapped:
#         swapped["Swapped_dimensions"] = True
#         return swapped
#     else:
#         return None

In [11]:
# import json

def classify_profile(cs, json_path, tol_dim=1.0, tol_area=0.05):
    """
    Attempts to classify a structural profile by matching its dimensions and area.
    Tries both the original and swapped width/height to detect if a 90° rotation is needed.
    """
    with open(json_path) as f:
        lib = json.load(f)

    def try_match(height, width):
        best, best_score = None, float('inf')
        for cat, ents in lib.items():
            for name, info in ents.items():
                dh = abs(height - info["height"])
                dw = abs(width  - info["width"])
                if dh > tol_dim or dw > tol_dim:
                    continue
                ea = abs(cs["area"] - info["csa"]) / info["csa"]
                if ea > tol_area:
                    continue
                el = abs(cs["length"] - info.get("length", cs["length"])) / info.get("length", cs["length"])
                score = dh + dw + ea * 100 + el * 100
                if score < best_score:
                    best_score = score
                    best = {
                        **info,
                        "Designation": name,
                        "Category": cat,
                        "Measured_height": height,
                        "Measured_width": width,
                        "Measured_area": cs["area"],
                        "Measured_length": cs["length"],
                        "Match_score": score,
                        "Profile_type": info["code_profile"],
                        "json_height": info["height"],
                        "json_width": info["width"],
                        "json_csa": info["csa"],
                    }
        return best, best_score

    # Try both normal and swapped width/height
    original, score1 = try_match(cs["span_web"], cs["span_flange"])
    swapped, score2  = try_match(cs["span_flange"], cs["span_web"])

    if original and (not swapped or score1 <= score2):
        original["Requires_rotation"] = False
        return original
    elif swapped:
        swapped["Requires_rotation"] = True
        return swapped
    else:
        return None


In [12]:
def print_result_table(result, precision=2):
    """
    Pretty print classification result as a one-row DataFrame with rounded floats.
    """
    if not result:
        print("No classification result.")
        return

    df = pd.DataFrame([result])

    # Round float values
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].round(precision)

    display(df.transpose().rename(columns={0: "Value"}))


In [13]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop

def compute_section_area(shape):
    props = GProp_GProps()
    brepgprop.SurfaceProperties(shape, props)
    return props.Mass()

def compute_axis_aligned_bounding_box(shape):
    """
    Computes a world-aligned bounding box (AABB) and includes true section area.
    """
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()

    # Half-sizes
    hx = (xmax - xmin) / 2
    hy = (ymax - ymin) / 2
    hz = (zmax - zmin) / 2

    center = gp_Pnt((xmin + xmax)/2, (ymin + ymax)/2, (zmin + zmax)/2)
    
    # Define frame as default world axes
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)
    ax3 = gp_Ax3(center, dir_z, dir_x)

    # Compute actual section area
    csa = compute_section_area(shape)

    return {
        "frame": ax3,
        "center": center,
        "dir_x": dir_x,
        "dir_y": dir_y,
        "dir_z": dir_z,
        "half_x": hx,
        "half_y": hy,
        "half_z": hz,
        "extents": [2*hx, 2*hy, 2*hz],
        "section_area": csa
    }


In [14]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt

def make_origin_marker(point=gp_Pnt(0, 0, 0), radius=5.0):
    """
    Creates a sphere centered at the global origin for visualization.
    """
    return BRepPrimAPI_MakeSphere(point, radius).Shape()


In [15]:
from OCC.Core.gp import gp_Pnt, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

def make_arrow(start: gp_Pnt, direction: gp_Dir, length=100.0):
    """
    Creates a line-shaped arrow from a start point in the given direction.
    """
    vec = gp_Vec(direction) * length
    end = gp_Pnt(start.XYZ())
    end.Translate(vec)
    return BRepBuilderAPI_MakeEdge(start, end).Shape()


In [16]:
from OCC.Core.gp import gp_Ax3, gp_Dir, gp_Pnt, gp_Trsf, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def align_obb_to_dstv_frame(shape, obb_frame: gp_Ax3, extents):
    """
    Aligns a solid from its OBB frame into the DSTV frame:
    X = length, Y = height, Z = width.
    Assumes axes are already swapped if needed based on profile classification.
    """
    # Unpack OBB origin and axis directions
    origin = obb_frame.Location()
    x_dir = obb_frame.XDirection()
    y_dir = obb_frame.YDirection()
    z_dir = obb_frame.Direction()  # Z axis of OBB frame

    # Create corrected local frame with updated axis mapping
    corrected_frame = gp_Ax3(origin, z_dir, x_dir)  # Z = up, X = beam axis

    # Create target DSTV frame (global axes)
    global_frame = gp_Ax3(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1), gp_Dir(1, 0, 0))  # Z up, X forward

    trsf = gp_Trsf()
    trsf.SetDisplacement(corrected_frame, global_frame)

    solid_transformed = BRepBuilderAPI_Transform(shape, trsf, True).Shape()
    return solid_transformed, trsf


In [17]:
# from OCC.Core.Bnd import Bnd_Box
# from OCC.Core.BRepBndLib import brepbndlib
# from OCC.Core.gp import gp_Pnt
# import numpy as np

# def compute_obb_geometry(aligned_shape):
#     """
#     Compute extents and axes of a shape aligned in its own OBB frame.
#     Returns length, height, width and the direction vectors.
#     """
#     # Step 1: Get the bounding box in the aligned frame
#     aabb = Bnd_Box()
#     brepbndlib.Add(aligned_shape, aabb)
#     xmin, ymin, zmin, xmax, ymax, zmax = aabb.Get()

#     # Step 2: Compute dimensions
#     length = xmax - xmin
#     height = ymax - ymin
#     width = zmax - zmin

#     # Step 3: Set direction vectors of the OBB (aligned axes)
#     # Since shape is aligned, these are just world axes
#     dir_x = np.array([1.0, 0.0, 0.0])  # X axis
#     dir_y = np.array([0.0, 1.0, 0.0])  # Y axis
#     dir_z = np.array([0.0, 0.0, 1.0])  # Z axis

#     # Step 4: Compute OBB center in local (aligned) coordinates
#     center = gp_Pnt(
#         (xmin + xmax) / 2,
#         (ymin + ymax) / 2,
#         (zmin + zmax) / 2
#     )

#     return {
#         "extents": [length, height, width],
#         "dir_x": dir_x,
#         "dir_y": dir_y,
#         "dir_z": dir_z,
#         "center": center
#     }


In [18]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.gp import gp_Pnt, gp_Dir

def compute_obb_geometry(aligned_shape):
    """
    Compute extents and axes of a shape aligned in its own OBB frame.
    Returns length, height, width and the direction vectors.
    """
    # Step 1: Get the bounding box in the aligned frame
    aabb = Bnd_Box()
    brepbndlib.Add(aligned_shape, aabb)
    xmin, ymin, zmin, xmax, ymax, zmax = aabb.Get()

    # Step 2: Compute dimensions
    length = xmax - xmin
    height = ymax - ymin
    width = zmax - zmin

    # Step 3: Use world axes since shape is aligned
    dir_x = gp_Dir(1, 0, 0)
    dir_y = gp_Dir(0, 1, 0)
    dir_z = gp_Dir(0, 0, 1)

    # Step 4: Compute center
    center = gp_Pnt(
        (xmin + xmax) / 2,
        (ymin + ymax) / 2,
        (zmin + zmax) / 2
    )

    return {
        "aligned_extents": [length, height, width],
        "aligned_dir_x": dir_x,
        "aligned_dir_y": dir_y,
        "aligned_dir_z": dir_z,
        "aligned_center": center
    }


In [19]:
def compute_section_area(solid):
    props = GProp_GProps()
    brepgprop.VolumeProperties(solid, props)
    volume = props.Mass()

    bbox = Bnd_Box()
    brepbndlib.Add(solid, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    length = xmax - xmin  # assuming length is X after alignment

    area = volume / length if length > 0 else 0
    return area


In [20]:
if __name__ == "__main__":

    step_path = "../data/0444-1 ANGLED.step"
    # step_path = "../data/ncTest.step"
    # step_path = "../data/TestEA.step"
    # step_path = "../data/TestEAMirror.step"
    # step_path = "../data/TestUEA.step"
    # step_path = "../data/TestUEAMirror.step"
    # step_path = "../data/TestPFC.step"
    shape_orig = load_step(step_path)


    # STEP 1: Align to major geometry (robust PCA/face-based)
    primary_aligned_shape, trsf, cs, largest_face, dir_x, dir_y, dir_z = robust_align_solid_from_geometry(shape_orig)


    # STEP 2: Compute OBB geometry
    obb_geom = compute_obb_geometry(primary_aligned_shape)
    aligned_extents = obb_geom["aligned_extents"]  # [length, height, width]
    aligned_center = obb_geom["aligned_center"]
    aligned_dir_x = obb_geom["aligned_dir_x"]
    aligned_dir_y = obb_geom["aligned_dir_y"]
    aligned_dir_z = obb_geom["aligned_dir_z"]

    # STEP 3: Compute section area
    section_area = compute_section_area(primary_aligned_shape)

    # STEP 4: Build classification signature
    cs = {
        "span_web": aligned_extents[1],
        "span_flange": aligned_extents[2],
        "area": section_area,
        "length": aligned_extents[0]
    }

    print("Classification input:")
    print("  span_web   =", cs["span_web"])
    print("  span_flange=", cs["span_flange"])
    print("  area       =", cs["area"])
    print("  length     =", cs["length"])

    # STEP 5: Match against JSON library
    profile_match = classify_profile(cs, json_path, tol_dim=1.0, tol_area=5)
    if profile_match is None:
        raise ValueError("❌ No matching profile found")

    # STEP 6: Handle swapped dimensions
    swap_axes = profile_match.get("Swapped_dimensions", False)
    if swap_axes:
        print("🔁 Swapping height/width axes to match profile classification")
        dir_y, dir_z = dir_z, dir_y
        extents[1], extents[2] = extents[2], extents[1]

    # STEP 7: Compute DSTV origin
    origin_local = compute_dstv_origin(aligned_center, aligned_extents, dir_x, dir_y, dir_z)

    # STEP 8: Align OBB to global DSTV frame using frame transform
    # Construct gp_Ax3 from origin and axis directions
    obb_frame = gp_Ax3(
        origin_local,
        dir_y,  # Y axis becomes "main" direction (Z is inferred from X × Y)
        dir_x   # X axis as the reference direction
    )

    aligned_shape, trsf = align_obb_to_dstv_frame(
        primary_aligned_shape,
        obb_frame,
        aligned_extents,
    )
    
    # STEP 9: Visual debugging — origin and axes
    origin_marker = make_origin_marker(gp_Pnt(0, 0, 0))
    arrow_x = make_arrow(gp_Pnt(0, 0, 0), gp_Dir(1, 0, 0))  # X = length
    arrow_y = make_arrow(gp_Pnt(0, 0, 0), gp_Dir(0, 1, 0))  # Y = height
    arrow_z = make_arrow(gp_Pnt(0, 0, 0), gp_Dir(0, 0, 1))  # Z = width

    visualize(origin_marker, viewer, color="RED")
    visualize(arrow_x, viewer, color="RED")
    visualize(arrow_y, viewer, color="GREEN")
    visualize(arrow_z, viewer, color="BLUE")
    visualize(aligned_shape, viewer, color="PSS_DARK_BLUE")
    visualize(make_origin_marker(origin_local), viewer, color="YELLOW")


    print_result_table(profile_match)


Classification input:
  span_web   = 203.20000020000143
  span_flange= 203.60000020000035
  area       = 5856.529123512673
  length     = 2280.0000002


,Value
mass,46.1
height,203.2
width,203.6
csa,5870.0
web_thickness,7.2
flange_thickness,11.0
root_radius,10.2
toe_radius,0.0
code_profile,I
Designation,203x203x46
